In [ ]:
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import math
import datetime as dt
import matplotlib as mpl
import matplotlib.pyplot as plt
import statistics as stats
import scipy
from scipy.stats.stats import pearsonr
from matplotlib import style
import warnings
import seaborn as sns
import requests
import bs4 as bs
import os
import csv
from openpyxl.workbook import Workbook as wb
from openpyxl import load_workbook
from openpyxl.styles import Font
from openpyxl.utils.dataframe import dataframe_to_rows
from pylab import rcParams
from sklearn import linear_model
import statsmodels.api as sm
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
warnings.filterwarnings('ignore') 

In [ ]:
# Dates for backtest
#'2004-12-31', '2006-12-31', '2008-12-31', '2010-12-31', '2012-12-31', '2008-12-31'
#'2009-12-31', '2011-12-31', '2013-12-31', '2015-12-31', '2017-12-31', '2019-12-31'
#'2007-12-31', '2009-12-31', '2011-12-31', '2013-12-31', '2015-12-31', '2015-12-31'
#'2007-12-20', '2009-12-20', '2011-12-20', '2013-12-20', '2015-12-20', '2015-12-20'
#'2014-01-20', '2016-01-20', '2018-01-20', '2020-01-20'

In [ ]:
start = input("Start Date ('1998-12-31'): ")
end = input("End Date ('2002-12-31'): ")
gamma_end = input("Gamma End Date ('2001-12-31'): ")
start_er =  input("Start Returns('2001-12-20'): ")
new_end = input("End for ROW: ")
year_excess_return = input("Years for Excess Return Test: ")

In [ ]:
data = pd.read_excel('stock_returns_data.xlsx')

In [ ]:
df = data[['Names Date','Ticker Symbol','Returns']]

In [ ]:
df = df[pd.to_numeric(df['Returns'], errors='coerce').notnull()]

In [ ]:
df['Date'] = df['Names Date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
df

In [ ]:
df['Returns']=df['Returns']*100
df1 = df[['Date','Ticker Symbol','Returns']]
df1['Date'] = df1['Date'].map(lambda x: 100*x.year + x.month)
df1['Date'] = df1['Date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m'))
df1

In [ ]:
df1 = df1[(df1['Date'] > start) & (df1['Date'] < end)]
df1

In [ ]:
df1 = df1.dropna(subset=['Ticker Symbol'])

In [ ]:
fama = pd.read_excel('FF_Research_Data_Factors.xlsx')
fama = fama[['Date','Mkt-RF','RF']]
fama['Returns'] = fama['Mkt-RF']+fama['RF']
fama['Ticker Symbol']='SP500'
fama1 = fama[['Date','Ticker Symbol','Returns','RF']]
fama1['Date'] = fama1['Date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m'))
fama1

In [ ]:
dfAll = pd.concat([df1,fama1])
dfAll = dfAll.reset_index()
dfAll = dfAll[(dfAll['Date'] > start) & (dfAll['Date'] < end)]
dfAll = dfAll[['Date','Ticker Symbol','Returns','RF']]
row1 = dfAll.tail(60)
#row1 = dfAll.tail(132)
row1 = row1[(row1['Date'] > start) & (row1['Date'] < gamma_end)]
row1 = row1['RF'].values
#row2 = dfAll.tail(48)
row2 = dfAll.tail(24)
row2 = row2['RF'].values
row3 = dfAll.tail(60)['RF'].values
#row3 = dfAll.tail(132)['RF'].values

In [ ]:
dfAll.tail(24)

In [ ]:
row2

In [ ]:
stock_count = dfAll.groupby('Ticker Symbol')['Returns'].count()
stock_count = stock_count[stock_count == 60]
#stock_count = stock_count[stock_count == 132]
stocks = [*stock_count.keys()]
stocks

In [ ]:
dfAll1 = dfAll.loc[dfAll['Ticker Symbol'].isin(stocks)]
dfAll1

In [ ]:
df_row = web.DataReader('VEMAX','yahoo',start,new_end)
df_row = df_row.reset_index()
df_row = df_row[['Date','Adj Close']]
df_row = df_row.resample('M', on='Date').last()
df_row = df_row[['Adj Close']]
df_row['Monthly Return'] = df_row['Adj Close'].pct_change()*100
df_row['ROW']=df_row['Monthly Return']
df_row = df_row[['ROW']]
df_row = df_row.reset_index()
df_row = df_row[(df_row['Date'] > start) & (df_row['Date'] < new_end)]
df_row = df_row.set_index('Date')
df_row

In [ ]:
df_row['RiskFree']=row3
df_row['ExcessReturn']=df_row['ROW']-df_row['RiskFree']
row = df_row['ExcessReturn'].values
row

In [ ]:
###Beta calculations###
###Want monthly return beta from 2015 to 2017###
dfAll1.reset_index()
dfBeta = dfAll1[(dfAll1['Date'] > start) & (dfAll1['Date'] < gamma_end)]
dfBeta = dfBeta.reset_index()
dfBeta = dfBeta[['Date','Ticker Symbol','Returns','RF']]
dfBeta

In [ ]:
#panel data
symbols = dfBeta.groupby('Ticker Symbol')
market_df = symbols.get_group('SP500')
market_df = market_df[['Date','Returns']].set_index('Date')
table = dfBeta.pivot(index='Date',columns='Ticker Symbol',values='Returns')  
table

In [ ]:
table['RiskFree'] = row1
table['ROW'] = row[:36]
#table['ROW'] = row[:84]
table1 = table.reset_index()
table1

In [ ]:
s = table1.columns[1:]
s

In [ ]:
beta_df = []
for stock in s:
    table1[stock] = table1[stock]-table1['RiskFree']
    beta_df.append({'Date':table1['Date'], 'Ticker':stock, 'ExcessReturn': table1[stock]})
beta_df = pd.DataFrame(beta_df)
beta_df1 = beta_df.set_index(['Ticker']).apply(pd.Series.explode).reset_index()

In [ ]:
beta_df2 = beta_df1.pivot(index='Date',columns='Ticker',values='ExcessReturn')
beta_df2

In [ ]:
y = beta_df2[['A']].values
x = beta_df2[['SP500', 'ROW']] 
regr = linear_model.LinearRegression()
regr.fit(x, y)
model = sm.OLS(y.astype(float), x.astype(float)).fit()
beta = model.params
beta

In [ ]:
equities = beta_df2.columns
main_df = []
for equity in equities:
    y = beta_df2[[equity]].values
    x = beta_df2[['SP500', 'ROW']] 
    regr = linear_model.LinearRegression()
    regr.fit(x, y)
    model = sm.OLS(y.astype(float), x.astype(float)).fit()
    predictions = model.predict(x) 
    beta = regr.coef_
    main_df.append({'Ticker':equity, 'Beta': beta})
main_df = pd.DataFrame(main_df)
main_df

In [ ]:
equities = beta_df2.columns
main_df = []
for equity in equities:    
    y = beta_df2[[equity]].values
    x = beta_df2[['SP500', 'ROW']]
    regr = linear_model.LinearRegression()
    regr.fit(x, y)
    model = sm.OLS(y.astype(float), x.astype(float)).fit()
    beta = model.params 
    main_df.append({'Ticker':equity, 'Beta': beta[0], 'Gamma': beta[1]})
main_df = pd.DataFrame(main_df)
#main_df.to_excel('gamma_tickers.xlsx')

In [ ]:
main_df1 = main_df.set_index('Ticker')
#main_df1.to_excel('beta_regression.xlsx')

In [ ]:
row1

In [ ]:
df_stock_returns = dfAll1[(dfAll1['Date'] > start_er) & (dfAll1['Date'] < end)]
stock_annual_returns = df_stock_returns.loc[df_stock_returns['Ticker Symbol'].isin(stocks)]
stock_returns = stock_annual_returns.pivot(index='Date',columns='Ticker Symbol',values='Returns')
stock_returns['Risk Free Rate'] = row2
stock_returns = stock_returns.reset_index()
z = stock_returns.columns
z = z[1:]
stock_returns_previous = []
for item in z:
    stock_returns[item] = stock_returns[item]-stock_returns['Risk Free Rate']
    stock_returns_previous.append({'Ticker':item, 'Excess Return': stock_returns[item].sum()})
stock_returns_previous = pd.DataFrame(stock_returns_previous)
stock_returns_previous

In [ ]:
#divide this by the number of years we tracked excess returns for to get the average annual excess return
stock_returns_previous['Annual Excess Return'] = stock_returns_previous['Excess Return']/int(year_excess_return)
excess_return_df = stock_returns_previous[['Ticker', 'Annual Excess Return']].set_index('Ticker')

In [ ]:
main1 = pd.concat([main_df1,excess_return_df],axis=1)
main1.index.names = ['Ticker']
main = main1
main = main.reset_index()
main.to_excel('gamma_tickers.xlsx')

In [ ]:
###Go to mrkt cap weight portfolio to finish the weights to transport to excel

In [ ]:
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import math
import datetime as dt
from openpyxl.workbook import Workbook as wb
from openpyxl import load_workbook
from openpyxl.styles import Font
from openpyxl.utils.dataframe import dataframe_to_rows 

In [ ]:
date_mktcap = input("Date (20071230): ")
yahoo_date = input("Date: ('2007-12-30'): ")

In [ ]:
shrout = pd.read_excel('shares_outstanding.xlsx')

In [ ]:
shrout.head(60)

In [ ]:
main = pd.read_excel('gamma_tickers.xlsx')

In [ ]:
out = shrout[shrout['Names Date'] == int(date_mktcap)]
shares = []
for ticker in out['Ticker Symbol']:
    shares.append(ticker)
shares = pd.DataFrame({'Ticker1':shares})
ticker_combined = pd.concat([shares,main['Ticker']],axis=1)
out

In [ ]:
match = ticker_combined['Ticker1'].isin(ticker_combined['Ticker'])
match = ticker_combined[match]
matches = []
for ticker in match['Ticker']:
    matches.append(ticker)
matches = pd.DataFrame({'Matching Tickers':matches})
matches = matches.dropna()
sorted_matches = matches.set_index('Matching Tickers')
sorted_matches = sorted_matches.sort_index(ascending = True)
sorted_matches = sorted_matches.reset_index()
sorted_matches

In [ ]:
failed = []
passed = []
data_mktcap = pd.DataFrame()
for ticker in sorted_matches['Matching Tickers'].values:
    try:
        data[ticker] = web.DataReader(ticker, data_source= "yahoo", start = yahoo_date, end=yahoo_date)["Adj Close"]
        passed.append(ticker)
    except (IOError, KeyError):
        msg = 'Failed to read symbol: {0!r}, replacing with NaN.'
        failed.append(ticker)

In [ ]:
#switching it to another name and flipping the table
adj_close = data_mktcap
#flipping the table 
adj_close = adj_close.T

In [ ]:
adj_close.index.names = ['Ticker']
adj_close.columns = ['Adj Close']
adj_close

In [ ]:
adj_close.index.names = ['Ticker']
adj_close = adj_close.reset_index()

In [ ]:
shrout1 = shrout.loc[shrout['Ticker Symbol'].isin(adj_close['Ticker'].values)]
shrout2 = shrout1[shrout1['Names Date'] == int(date)]
shrout2 = shrout2.set_index('Ticker Symbol')
shrout2.index.names = ['Ticker']
shrout2 = shrout2[['Names Date','Shares Outstanding']]
shrout2 = shrout2.sort_index(ascending = True)

In [ ]:
adj_close = adj_close.set_index('Ticker')
adj_close

In [ ]:
adj_close.sort_values('Adj Close')

In [ ]:
cap = pd.concat([shrout2,adj_close],axis=1)
cap['Market Cap'] = cap['Shares Outstanding']*cap['Adj Close']
cap = cap[['Market Cap']]
cap['Weight'] = cap['Market Cap']/cap['Market Cap'].values.sum()
cap

In [ ]:
cap.sort_values("Market Cap", ascending = False)

In [ ]:
cap.to_excel('weights.xlsx')

In [ ]:
weights = pd.read_excel('weights.xlsx')

In [ ]:
weights

In [ ]:
weights = weights.set_index('Ticker')
weights.index.names = ['Ticker']
weights

In [ ]:
whole = pd.concat([main1,weights],axis=1)
whole = whole.dropna()
whole['Beta Weighted'] = whole['Beta']*whole['Weight']
whole['Gamma Weighted'] = whole['Gamma']*whole['Weight']
whole['Excess Return Weighted'] = whole['Annual Excess Return']*whole['Weight']
whole_gamma = whole.sort_values('Gamma',ascending = False)
whole_beta = whole.sort_values('Beta',ascending = False)
whole_weight = whole.sort_values('Weight',ascending = False)
whole_gamma_weight = whole.sort_values('Gamma Weighted',ascending = False)
whole_beta_weight = whole.sort_values('Beta Weighted',ascending = False)
whole_excess_return_weight = whole.sort_values('Excess Return Weighted',ascending = False)
#whole_excess_return_weight
whole_gamma_weight

In [ ]:
size = whole.sort_values('Market Cap',ascending = False)
size

In [ ]:
s = size[238:][['Beta','Gamma','Annual Excess Return']].sort_values('Gamma',ascending = False)
m = size[119:238][['Beta','Gamma','Annual Excess Return']].sort_values('Gamma',ascending = False)
l = size[:119][['Beta','Gamma','Annual Excess Return']].sort_values('Gamma',ascending = False)

In [ ]:
###gamma of each portfolio sorted by size (mkt-cap)
group_size = 23
portfolios = [s.iloc[i:i+group_size] for i in range(0,len(s)-group_size+1,group_size)]   
small_portfolio_gammas = []
for portfolio in portfolios:
    g = (portfolio['Gamma'].mean())
    b = (portfolio['Beta'].mean())
    r = (portfolio['Annual Excess Return'].mean())
    small_portfolio_gammas.append({'Gamma':g, 'Beta':b, 'Annual Excess Return':r})
small_portfolio_gammas = pd.DataFrame(small_portfolio_gammas)
small_portfolio_gammas

In [ ]:
###gamma of each portfolio sorted by size (mkt-cap)
group_size = 23
portfolios = [m.iloc[i:i+group_size] for i in range(0,len(m)-group_size+1,group_size)]   
medium_portfolio_gammas = []
for portfolio in portfolios:
    g = (portfolio['Gamma'].mean())
    b = (portfolio['Beta'].mean())
    r = (portfolio['Annual Excess Return'].mean())
    medium_portfolio_gammas.append({'Gamma':g, 'Beta':b, 'Annual Excess Return':r})
medium_portfolio_gammas = pd.DataFrame(medium_portfolio_gammas)
medium_portfolio_gammas

In [ ]:
###gamma of each portfolio sorted by size (mkt-cap)
group_size = 23
portfolios = [l.iloc[i:i+group_size] for i in range(0,len(l)-group_size+1,group_size)]   
large_portfolio_gammas = []
for portfolio in portfolios:
    g = (portfolio['Gamma'].mean())
    b = (portfolio['Beta'].mean())
    r = (portfolio['Annual Excess Return'].mean())
    large_portfolio_gammas.append({'Gamma':g, 'Beta':b, 'Annual Excess Return':r})
large_portfolio_gammas = pd.DataFrame(large_portfolio_gammas)
large_portfolio_gammas

In [ ]:
###gamma of each portfolio sorted from highest to lowest beta
group_size = 196
portfolios = [whole_gamma_weight.iloc[i:i+group_size] for i in range(0,len(whole_gamma_weight)-group_size+1,group_size)]   
portfolio_gammas = []
for portfolio in portfolios:
    g = (portfolio['Gamma Weighted'].sum())
    b = (portfolio['Beta Weighted'].sum())
    r = (portfolio['Excess Return Weighted'].sum())
    portfolio_gammas.append({'Gamma Weighted':g, 'Beta Weighted':b, 'Excess Return Weighted':r})
portfolio_gammas = pd.DataFrame(portfolio_gammas)
portfolio_gammas

In [ ]:
###2008-2012
whole_beta_weight
###beta of each portfolio sorted from highest to lowest beta
group_size = 196
portfolios = [whole_beta_weight.iloc[i:i+group_size] for i in range(0,len(whole_beta_weight)-group_size+1,group_size)]   
portfolio_betas = []
for portfolio in portfolios:
    g = (portfolio['Gamma Weighted'].sum())
    b = (portfolio['Beta Weighted'].sum())
    r = (portfolio['Excess Return Weighted'].sum())
    portfolio_betas.append({'Gamma Weighted':g, 'Beta Weighted':b, 'Excess Return Weighted':r})
portfolio_betas = pd.DataFrame(portfolio_betas)
portfolio_betas

In [ ]:
###SP500 excess return for this period
excess_return_df.iloc[3495]

In [ ]:
###2009-2019
plt.scatter(x = portfolio_gammas['Beta Weighted'], y=portfolio_gammas['Excess Return Weighted'], c=portfolio_gammas['Gamma Weighted'])
#plt.scatter(x = portfolio_betas['Beta Weighted'], y=portfolio_betas['Excess Return Weighted'], c='r')
plt.title('Mkt Cap Weighted Gamma-Sorted Portfolios to Annual Excess Returns')
plt.xlabel('Portfolio Mkt Cap Weighted Beta')
plt.ylabel('Annual Excess Return (%)')
x = np.linspace(-.5,.5,2)
#x = np.linspace(-10,10,20)
y = 6.18*x
plt.plot(x, y, '-b', label='SP500 excess annual return')
plt.legend(loc='upper left')
plt.grid()
plt.colorbar()
plt.show()

In [ ]:
#plt.scatter(x = portfolio_gammas['Beta Weighted'], y=portfolio_gammas['Excess Return Weighted'], c=portfolio_gammas['Gamma Weighted'])
plt.scatter(x = portfolio_betas['Beta Weighted'], y=portfolio_betas['Excess Return Weighted'], c='r')
plt.title('Mkt Cap Weighted Beta-Sorted Portfolios to Annual Excess Returns')
plt.xlabel('Portfolio Mkt Cap Weighted Beta')
plt.ylabel('Annual Excess Return (%)')
x = np.linspace(-.5,.5,2)
y = 16.355*x
plt.plot(x, y, '-b', label='SP500 excess annual return')
plt.legend(loc='upper left')
plt.grid()
#plt.colorbar()
plt.show()

In [ ]:
#just to test the results to see if the excess returns are correct
whole_gamma_weight.head(50)